Pytorch 先动态搭建计算图，然后运行反向传播进行自动求导，下面就介绍一下 Pytorch 的计算图和自动求导机制：

## 1. 计算图
计算图是用来描述运算的有向无环图，有两个主要元素：结点和边，结点表示张量数据，边表示运算。

计算图采用链式法则进行微分求导。

计算图示例：y = (x+w) * (w+1)

![](./图1.png)

用代码实现该计算图示例：

In [1]:
import torch

w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)   
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()

# 2. tensor 四个和计算图相关的属性
tensor 有 8 个属性，分别是 data, dtype, shape, device, requires_grad, grad, grad_fn, is_leaf，下面结合代码介绍后四个和计算图相关的属性：

## 1) requires_grad：指示张量是否需要计算梯度

In [2]:
# 查看张量是否需要计算梯度
w.requires_grad, x.requires_grad, a.requires_grad, b.requires_grad, y.requires_grad

(True, True, True, True, True)

也许大家会有疑问，在创建 a, b, y 时，并没有设置 requires_grad=True，为啥上面代码他们的输出结果也都为 True 呢？因为 a, b, y 都是依赖于叶子结点的结点，而依赖于依赖于叶子结点的结点的 requires_grad 属性默认值为 True。

## 2) is_leaf：指示张量是否为叶子结点

叶子结点：用户创建的张量称为叶子结点，是整个计算图的根基，如第一张图的 x 和 w；在反向传播中，所有梯度的计算都依赖于叶子结点。

In [3]:
# 查看张量是否是叶子结点
w.is_leaf, x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf

(True, True, False, False, False)

## 3) grad：指示张量的梯度值

注意：在求解梯度的反向传播结束之后，非叶子结点的梯度占用的内存会被释放掉，叶子结点的梯度占用的内存会被保留。

In [4]:
# 查看张量的梯度
w.grad, x.grad, a.grad, b.grad, y.grad

C:\Users\24606\AppData\Local\Temp\ipykernel_8600\2984833489.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  w.grad, x.grad, a.grad, b.grad, y.grad


(tensor([5.]), tensor([2.]), None, None, None)

在上面代码运行结果中，由于 a, b, y 的梯度占用的内存已经被释放掉，所以 a, b, y 的梯度显示为 None。
如果想保留某些非叶子结点的梯度，而不是在反向传播之后释放掉其占用的内存，可以使用 retain_grad() 设置。

In [5]:
# retain_grad() 使用示例
# 保留非叶子结点 a 的梯度
a = torch.add(w, x) 
a.retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward()
# print(w.grad)

# 查看张量的梯度
w.grad, x.grad, a.grad, b.grad, y.grad

C:\Users\24606\AppData\Local\Temp\ipykernel_8600\289658673.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  w.grad, x.grad, a.grad, b.grad, y.grad


(tensor([10.]), tensor([4.]), tensor([2.]), None, None)

在上面代码运行结果中，非叶子结点 a 的梯度被保留了。

### 3-1) 梯度清零问题

在上面代码的运行结果中，我们发现 w.grad 和 x.grad 的值从 5 和 2 变成了 10 和 4，原因在于 autograd 求解的叶子结点的梯度占用的内存会被保留，在多次反向传播过程中会逐渐累加，不会自动清零。

但是需要注意的是，对于非叶子结点，即使设置了 a.retain_grad()，在多次反向传播过程中，其梯度值并不会逐渐累加，一直都是同一个值。

如果我们使用完叶子结点的梯度之后想将叶子结点的梯度清零，可以设置 w.grad.zero_()

In [6]:
print(w.grad, x.grad)
print(id(w), id(x))

w.grad.zero_()
x.grad.zero_()
print(w.grad, x.grad)
print(id(w), id(x))

tensor([10.]) tensor([4.])
3147837759792 3147837759632
tensor([0.]) tensor([0.])
3147837759792 3147837759632


上面梯度清零操作用到的 w.grad.zero_()，函数后为啥紧跟一个下划线呢？

### 3-2) 原位操作
函数后紧跟一个下划线表示原位（in-place）操作，即在原内存地址对数据进行修改。

下面通过代码解释原位操作：

In [7]:
m = torch.ones(1,)
print(m, id(m))

# 运算 1 
m.add_(1)
print(m, id(m))

# 运算 2
m += 1
print(m, id(m))

# 运算 3
m = m + 1
print(m, id(m))

tensor([1.]) 3146323276400
tensor([2.]) 3146323276400
tensor([3.]) 3146323276400
tensor([4.]) 3147837005392


我们可以发现，前 2 种运算结果的内存地址都是和初始地址一样的，两者都是原位操作；

注意：+= 和 先+后赋值不是等同的，后者创建了新的内存地址。

## 4) grad_fn：记录张量创建时所用的方法（函数）

In [8]:
# 查看张量创建时所用的方法（函数）
w.grad_fn, x.grad_fn, a.grad_fn, b.grad_fn, y.grad_fn

(None,
 None,
 <MulBackward0 at 0x2dc8f7af3d0>)

w 和 x 是我们直接定义的，没有经过任何方法/函数去创建，所以它俩的 grad_fn 为 None。

# 3. 动态图和静态图

动态图：边运算边搭建计算图，优点是灵活；

静态图：先搭建计算图，后运算，优点是高效；

Pytorch 的计算图采用的是动态图机制；

TensorFlow 的计算图采用的是静态图机制。

示例：运行以下代码的过程就是搭建计算图的过程，运行一次代码就会创建一次计算图。

    w = torch.tensor([1.], requires_grad=True)
    x = torch.tensor([2.], requires_grad=True)
        
    a = torch.add(w, x)   
    b = torch.add(w, 1)
    y = torch.mul(a, b)

# 4. torch.autograd.backward() 自动求导

xxx.backward() 和 torch.autograd.backward() 是等价的，都是用于自动求取梯度，有 gradient, retain_graph, create_graph, inputs 四个参数：

inputs：用于求导的张量

create_graph：参考 torch.autograd.grad() 中的介绍

## 1) retain_graph：保存计算图

执行完反向传播之后，计算图将被释放，再次执行 y.backward() 将会报错，如果想再次执行 y.backward()，可以将 retain_graph 设置为 True，retain_graph 默认为 False。

In [9]:
a = torch.add(w, x) 
a.retain_grad()
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward(retain_graph=True)
print(w.grad)
w.grad.zero_()

y.backward()
print(w.grad)

tensor([5.])
tensor([5.])


## 2) gardient：设置多个梯度之间的权重

例如，以下示例中，w 的梯度值 = y0 对 w 的梯度值 * 权重 + y1 对 w 的梯度值 * 权重 

In [10]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)    
b = torch.add(w, 1)

y0 = torch.mul(a, b)   
y1 = torch.add(a, b) 

loss = torch.cat([y0, y1], dim=0)     
grad_tensors = torch.tensor([1., 2.])  # 创建多个梯度的权重张量

loss.backward(gradient=grad_tensors)  

print(w.grad)

tensor([9.])


# 5. torch.autograd.grad() 自动求导

torch.autograd.grad() 用来针对性地求取某个受关注张量的梯度，有 outputs, inputs, retain_graph, create_graph, grad_outputs 五个参数；

outputs：用于求导的张量

inputs：需要梯度的张量

retain_graph：参考 torch.autograd.backward() 中的介绍

grad_outputs：参考 torch.autograd.backward() 中对 gardient 的介绍

## 1) create_graph：创建导数的计算图
create_graph 设置为 True 时可以创建导数的计算图，导数的计算图用来对导数进行求导（高阶求导）。

代码示例：

In [11]:
x = torch.tensor([3.], requires_grad=True)
y = torch.pow(x, 2)

grad_1 = torch.autograd.grad(y, x, create_graph=True)  # grad_1 = dy/dx = 2x = 2 * 3 = 6
print(grad_1)  # 返回一个元组

# 对导数进行求导
grad_2 = torch.autograd.grad(grad_1[0], x)  # grad_2 = d(dy/dx)/dx = d(2x)/dx = 2
print(grad_2)

(tensor([6.], grad_fn=<MulBackward0>),)
(tensor([2.]),)
